In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np

pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(precision=6,threshold=5000,linewidth=200,suppress=True)

In [2]:
col_names = ["id","title","description","brand_id","brand_name","publisher_id", "publisher_name", "ean","parent_id","chunk_id","brick_id","unit", "category", "group", "subgroup"]
dtypes = {"id":object,"title":object,"description":object,
          "brand_id":object,"brand_name":object,
          "publisher_id":object, "publisher_name":object, 
          "ean":object,"parent_id":object,"chunk_id":object,"brick_id":object,
          "unit":object,"category":object,"group":object,"subgroup":object}

In [3]:
data_orig = pd.read_csv("part-m-00000.gz",sep="\001",header=None,names=col_names,dtype=dtypes)
data = data_orig

In [4]:
data = data[~data['unit'].isin(['UNKNOWN','Unknown'])]

In [5]:
data['id_known'] = data['brand_id'].notnull() * 1

/Users/svandenoord/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [6]:
data.iloc[20:25]

,id,title,description,brand_id,brand_name,publisher_id,publisher_name,ean,parent_id,chunk_id,brick_id,unit,category,group,subgroup,id_known
20,1000004006848117,mountain homecoming,NaN,NaN,NaN,1154711,Spring House Music Group,0617884222024,42789767,30010233,10001137,Toys & Electronics,Computer & Games,Games Software,Games Software,0
21,1000004007400074,bisolvon drank kinderen 125 ml 1 stuk,bisolvon drank voor kinderen werkt slijmoploss...,7605984,Bisolvon,NaN,NaN,8711642000007,42920467,80007098,10000876,Lifestyle,Health & Intimacy,Health,Hooikoorts & Allergieen,1
22,1000004007406991,traay polderhoning vloeibaar 100 gram honing,honing is een product van de natuur uit het ge...,7452911,De Traay,NaN,NaN,8713406140233,42927376,80008269,10000468,Lifestyle,Health & Intimacy,Health,Voedingssupplementen BPH,1
24,1000004007478860,plug fischer s4 200st,er zullen maar weinig gebouwen zijn waar deze ...,8073659,Fischer,NaN,NaN,4006209501047,42959352,30015498,10006247,Home & Living,Home Improvement & Gardening,Home Improvement,Ironware,1
25,1000004008172167,nijntje blauw gebreid spenenketting,altijd de speen bij de hand met deze spenenket...,2552880,Nijntje,NaN,NaN,8713291001084,43003047,80006764,10005153,Toys & Electronics,Toys,Toys Core,Babyspeelgoed,1


In [7]:
data_full = data[data['brand_id'].notnull()]
data_null = data[data['brand_id'].isnull()]
data_full.reset_index(inplace=True,drop=True)
data_null.reset_index(inplace=True,drop=True)

In [8]:
# sommige titels zijn nan, die maak ik hier gewoon leeg
data_full['title'].iloc[data_full[data_full['title'].isnull()].index] = ""

/Users/svandenoord/anaconda3/lib/python3.5/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/svandenoord/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [9]:
data_full['group'].value_counts(ascending=False)

Mobile & Tablet Accessories                      340221
Home accessories Textile & Window Decorations    324796
Sport                                            160506
Home Improvement                                 132188
Baby Apparel                                     104289
Jewellery                                         93982
Toys Core                                         88975
Computer Accessories                              77230
Leisure                                           65110
Biking                                            50343
Pet Non-food                                      48027
Consumer Electronics Accessories                  45014
Hobby & Crafts                                    44963
Gardening                                         43290
Health                                            30984
Personal Care                                     28059
Intimacy                                          27770
Dining                                          

In [10]:
data_full[data_full['group']=='Games Consoles'][['title','description']].head(3)

,title,description
12843,playstation 3 60 gb,de nieuwe sensatie op het gebied van games en ...
12895,nintendo ds lite wit,de nintendo ds lite biedt je dezelfde krachtig...
12896,nintendo ds lite zwart,de nintendo ds lite biedt je dezelfde krachtig...


In [11]:
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

In [14]:
product_groups = data_full.groupby('group')

In [15]:
product_groups.get_group('Baby Home')

,id,title,description,brand_id,brand_name,publisher_id,publisher_name,ean,parent_id,chunk_id,brick_id,unit,category,group,subgroup,id_known
18432,1004004007093635,bumba knuffeldoekje,knuffeldoekje met bumba figuur voorzien van va...,2593796,Bumba,2163537,Studio 100,5414233010104,41012010,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
18483,1004004007093922,in de droomtuin igglepiggle dekentje,in the nightgarden dekentje aan dit lekker zac...,4444546,Hasbro,4444546,Hasbro,5010994325848,41012297,80002293,10002224,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
20331,1004004007690218,dikkie dik serviesset melamine meerkleurig,5delige dikkie dikserviesset bestaande uit bor...,3741773,Bambolino Toys,3741773,Bambolino Toys,8716473520014,41021304,80006465,10000490,Lifestyle,Baby,Baby Home,Baby Servies Bekers & Bestek,1
21267,1004004009504392,happy horse dikkie dik knuffeldoekje,dikkie dik de bekendste eigenwijze oranje kat ...,4408862,Happy Horse,4408862,Happy Horse,8711811075591,41024622,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
22143,1004004010504098,happy horse aap mo knuffeldoekje,dit vrolijke aap mo knuffeldoekje tutteldoekje...,4408862,Happy Horse,4408862,Happy Horse,8711811075881,41027028,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
22153,1004004010504132,happy horse schaap lammy knuffeldoekje,een lammy knuffeldoekje tutteldoekje om heerli...,4408862,Happy Horse,4408862,Happy Horse,8711811037841,41027062,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
22162,1004004010504177,happy horse beer boogy knuffeldoekje,een beige beer boogy knuffeldoekje tutteldoekj...,4408862,Happy Horse,4408862,Happy Horse,8711811060672,41027107,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
22364,1004004010605401,chicco knuffeldoekje slak,knuffeldoekje de ronde handgrepen zijn zacht e...,5329407,Chicco,3539959,Artsana,8003670957454,41027706,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
22701,1004004010686618,steiff hoppel haas knuffeldoek,steiff hoppel haas knuffeldoek is gemaakt van ...,4476953,Steiff,NaN,NaN,4001505237478,41028865,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1
22702,1004004010686682,sniffy maus schmusetuch blau 30 cm,steiff ean 237614 baby knuffeldoek sniffy muis...,4476953,Steiff,NaN,NaN,4001505237614,41028929,80006824,10005142,Lifestyle,Baby,Baby Home,Baby Dekentjes,1


In [16]:
groups_list = [group for group in product_groups.groups]

In [18]:
groups_list[0:5]

['Small Domestic Appliances',
 'Consumer Electronics Accessories',
 'Baby Hardware',
 'Major Domestic Appliances',
 'Home Improvement']

In [19]:
selected_groups = ['Beauty', 'Baby Home', 'Pet Food', 'Sunglasses']

In [23]:
for group in selected_groups:
    data = data_full[data_full['group'] == group]
    print(data[['group','title','brand_name']].head(3))
    print("\n")

        group                                              title brand_name
18365  Beauty                   puntenslijper topwrite wereldbol  geen merk
21302  Beauty                                        geodriehoek   Toppoint
45496  Beauty  blomus makeup spiegel primo wandmodel met 5 x ...     Blomus


           group                                       title      brand_name
18432  Baby Home                         bumba knuffeldoekje           Bumba
18483  Baby Home        in de droomtuin igglepiggle dekentje          Hasbro
20331  Baby Home  dikkie dik serviesset melamine meerkleurig  Bambolino Toys


           group                                           title brand_name
104947  Pet Food       velda vijvernatuurvoer vissenvoer130 gram      Velda
104948  Pet Food  velda krachtvoer rondett power food vissenvoer      Velda
104951  Pet Food      vt visvoeder stick 1500 ml 220g vissenvoer         VT


            group                             title brand_name
86994  Sunglass

In [30]:
data_full[data_full['group'] == 'Baby Home'].shape

(5797, 16)

In [29]:
from sklearn.cross_validation import train_test_split

In [43]:
from sklearn.metrics import accuracy_score

In [42]:
y_test.head(3)

,brand_id
1146492,10559072
1423279,3761565
71993,6764436


In [117]:
from multiprocessing import cpu_count
cpu_count = cpu_count()
print(cpu_count)

def build_model(category, data):
    data_temp = data[data['group'] == category]
    columns_X = ['title']
    columns_y = ['brand_id']
    X_train, X_test, y_train, y_test = train_test_split(data_temp[columns_X], data_temp[columns_y],random_state = 42)
    n_iter = np.ceil(10**6 / X_train.shape[0])
    n_iter = int(max(5,n_iter))
    print(category, X_train.shape[0], n_iter) 
    pipeline = Pipeline([
        ('tf_idf', TfidfVectorizer(strip_accents='unicode',ngram_range=(1,2),sublinear_tf=True)),
        ('sgd', SGDClassifier(loss='modified_huber', verbose=0, n_iter=n_iter, n_jobs=cpu_count-1))
    ])
    pipeline.fit(X_train.values.ravel(),y_train.values.ravel())
    accuracy = accuracy_score(pipeline.predict(X_test.values.ravel()), y_test)
    print(round(accuracy,3))
    return {"category":category, "model":pipeline, "accuracy":accuracy, "size_train":X_train.shape[0]}

8


In [106]:
all_models = {}
print("total nr of groups: {}".format(len(groups_list)))

for nr, group in enumerate(groups_list):
    print("now starting on {}: {}".format(nr,group))
    model = build_model(group, data_full)
    all_models[group] = model

now starting on: Small Domestic Appliances
Small Domestic Appliances 14315 70
0.868
now starting on: Consumer Electronics Accessories
Consumer Electronics Accessories 33760 30
0.898
now starting on: Baby Hardware
Baby Hardware 9196 109
0.911
now starting on: Major Domestic Appliances
Major Domestic Appliances 3411 294
0.881
now starting on: Home Improvement
Home Improvement 99141 11
0.897
now starting on: Baby Home
Baby Home 4347 231
0.886
now starting on: Computer Accessories
Computer Accessories 57922 18
0.89
now starting on: Dining
Dining 20142 50
0.881
now starting on: Outdoor Toys
Outdoor Toys 12043 84
0.852
now starting on: Home accessories Textile & Window Decorations
Home accessories Textile & Window Decorations 243597 5
0.958
now starting on: Pet Sport Equipment
Pet Sport Equipment 5084 197
0.949
now starting on: Gardening
Gardening 32467 31
0.85
now starting on: Health
Health 23238 44
0.938
now starting on: Personal Care
Personal Care 21044 48
0.879
now starting on: Baby Care

In [195]:
all_models['Beauty']['model']

Pipeline(steps=[('tf_idf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False))])

In [212]:
type(all_models)

dict

In [211]:
pwd

'/Users/svandenoord/Datascience/Merken2.0/AllBrands'

In [215]:
from sklearn.externals import joblib
joblib.dump(all_models, 'all_models_dict.pkl') 

['all_models_dict.pkl',
 'all_models_dict.pkl_01.npy',
 'all_models_dict.pkl_02.npy',
 'all_models_dict.pkl_03.npy',
 'all_models_dict.pkl_04.npy',
 'all_models_dict.pkl_05.npy',
 'all_models_dict.pkl_06.npy',
 'all_models_dict.pkl_07.npy',
 'all_models_dict.pkl_08.npy',
 'all_models_dict.pkl_09.npy',
 'all_models_dict.pkl_10.npy',
 'all_models_dict.pkl_11.npy',
 'all_models_dict.pkl_12.npy',
 'all_models_dict.pkl_13.npy',
 'all_models_dict.pkl_14.npy',
 'all_models_dict.pkl_15.npy',
 'all_models_dict.pkl_16.npy',
 'all_models_dict.pkl_17.npy',
 'all_models_dict.pkl_18.npy',
 'all_models_dict.pkl_19.npy',
 'all_models_dict.pkl_20.npy',
 'all_models_dict.pkl_21.npy',
 'all_models_dict.pkl_22.npy',
 'all_models_dict.pkl_23.npy',
 'all_models_dict.pkl_24.npy',
 'all_models_dict.pkl_25.npy',
 'all_models_dict.pkl_26.npy',
 'all_models_dict.pkl_27.npy',
 'all_models_dict.pkl_28.npy',
 'all_models_dict.pkl_29.npy',
 'all_models_dict.pkl_30.npy',
 'all_models_dict.pkl_31.npy',
 'all_models_di

In [199]:
import pickle

In [201]:
with open('all_models_beauty.pkl', 'wb') as file_name:
    pickle.dump(all_models['Beauty']['model'],file_name)

In [205]:
with open('all_models_beauty.pkl', 'rb') as file_name:
   all_models_beauty = pickle.load(file_name)

In [207]:
all_models_beauty.predict(['chanel chez homme','dolce sweet','dg','gabbana'])

array(['7819612', '7105536', '7105536', '7105536'], 
      dtype='<U8')

In [203]:
pwd

'/Users/svandenoord/Datascience/Merken2.0/AllBrands'

In [108]:
len(all_models)

46

In [109]:
len(groups_list)

46

In [119]:
scores = []
for nr, key in enumerate(all_models):
    scores.append([key, round(all_models[key]['accuracy'],3)])

In [152]:
pd.DataFrame(scores, columns=['category','accuracy']).sort_values(by='accuracy',ascending=False)

,category,accuracy
25,Navigation,0.98400
30,Sport,0.96000
42,Jewellery,0.95900
9,Home accessories Textile & Window Decorations,0.95800
34,Biking,0.95500
33,Baby Apparel,0.95200
10,Pet Sport Equipment,0.94900
45,Computer,0.94600
15,Watches,0.94300
17,Leisure,0.94200


In [125]:
all_models['Intimacy']

{'accuracy': 0.72792740890105145,
 'category': 'Intimacy',
 'model': Pipeline(steps=[('tf_idf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max_features=None, min_df=1,
         ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
 ...   penalty='l2', power_t=0.5, random_state=None, shuffle=True,
        verbose=0, warm_start=False))])}

array(['7452235'], 
      dtype='<U8')

In [138]:
brand_id_list = data_full['brand_id'].tolist()
brand_name_list = data_full['brand_name'].tolist()

In [139]:
brand_translator = {}
for i in range(len(brand_id_list)):
    if brand_translator.get(brand_id_list[i],None) is None:
        brand_translator[brand_id_list[i]] = brand_name_list[i]

In [209]:
pickle.dump(brand_translator, open("brand_translator2.pkl", "wb"))

In [186]:
for i in range(0,5):
    prediction = all_models['Navigation']['model'].predict([data_null[data_null['group'] == 'Navigation'].iloc[i]['title']])
    prediction_proba = all_models['Navigation']['model'].predict_proba([data_null[data_null['group'] == 'Navigation'].iloc[i]['title']])
    print(data_null[data_null['group'] == 'Navigation'].iloc[i]['id'])
    print(data_null[data_null['group'] == 'Navigation'].iloc[i]['title'])
    print(prediction.tolist()[0])
    print(brand_translator[prediction.tolist()[0]])
    print(np.max(prediction_proba))
    print("\n")

9200000048722773
zenec navigatie ze3726 fiat ducato
10548753
Zenec
0.57978627197


9200000052538616
onroute motorkaart benelux 20152016 msd
15159315
Car Gadgets. BV
0.568998565803


9200000052538618
onroute motorkaart westeuropa 20152016 msd
15159315
Car Gadgets. BV
0.318731305281


9200000056898673
achteruit rij camera
15159315
Car Gadgets. BV
0.937683642579


9200000058293027
snooper s8100 truckmate pro
10513573
Snooper
0.784018595508




In [191]:
data_null[data_null['group'] == 'Navigation'].iloc[0]['id']

'9200000048722773'

In [167]:
np.argsort(prediction_proba,axis=1)[0][::-1][0]

782

In [169]:
np.max(prediction_proba)

0.678824107183019

In [187]:
all_models['Intimacy']['model'].named_steps['sgd'].classes_[782]

'9435282'

In [189]:
50 // 3

16

In [190]:
1 % 3

1